In [1]:
import google.generativeai as genai
from google.colab import drive, userdata
import os
import json
import openpyxl
from openpyxl import Workbook
import PIL.Image

In [2]:
import re

In [3]:
def get_existing_image_ids(excel_file):
    """
    Retrieve the list of image IDs already present in the Excel file.

    Parameters:
    excel_file (str): Path to the Excel file.

    Returns:
    set: A set of image IDs present in the file.
    """
    if os.path.exists(excel_file):
        workbook = openpyxl.load_workbook(excel_file)
        sheet = workbook.active
        image_ids = {row[0].value for row in sheet.iter_rows(min_row=2, max_col=1)}
        return set(image_ids)
    return set()

In [4]:
def get_llm_response(file_name, img):

  generation_config = {
      "temperature": 1,
      "top_p": 0.95,
      "top_k": 40,
      "max_output_tokens": 8192,
      "response_mime_type": "text/plain",
  }

  model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
  )

  text_input = """You are provided with the invoice.
                Give the date and total amount from the receipt.
                Give the response in the following json format
                {{"image_id": "Jan_2025",
                "date": "dd/mm/yyyy",
                "Amount": "4000.50"}}
                Here the image_id is created using month and year of the date"""

  prompt = [
        text_input, img
  ]

  response = model.generate_content(prompt)
  return response.text


In [5]:
def process_invoices(file_name, folder_path, excel_file, json_data):

    print(json_data)
    # tt = input()
    # Define the Excel file name


    # if isinstance(json_data, str):
    #     json_data = json.loads(json_data)
    try:
        # Regular expression to extract JSON from the text
        json_match = re.search(r"\{.*\}", json_data, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)  # Get the JSON string
            json_data = json.loads(json_str)    # Convert to Python dictionary
        else:
            raise ValueError("No JSON found in the LLM output.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

    # Check if the Excel file exists
    if os.path.exists(excel_file):
        # Open the existing workbook
        workbook = openpyxl.load_workbook(excel_file)
        sheet = workbook.active
    else:
        # Create a new workbook and add headers
        workbook = Workbook()
        sheet = workbook.active
        sheet.title = "Invoices"
        sheet.append(["image_id", "date", "Amount"])

    # Add the JSON data to the Excel file
    sheet.append([file_name, json_data["date"], json_data["Amount"]])

    # Save the workbook
    workbook.save(excel_file)
    print(f"Data saved in {excel_file}")

In [10]:
def main():


  gemini_api_key = userdata.get("GEMINI_KEY")

  genai.configure(api_key=gemini_api_key)

  # Mount Google Drive
  drive.mount('/content/drive')

  # Define the folder path in your Google Drive
  folder_path = "/content/drive/My Drive/Invoices"  # Replace with your folder path

  excel_file = os.path.join(folder_path, "invoices.xlsx")
  output_path = excel_file


  existing_image_ids = []
  existing_image_ids = get_existing_image_ids(excel_file)
  print("Existing image IDs:", existing_image_ids)

  for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpeg") or file_name.endswith(".jpg"):

            if file_name in existing_image_ids:
                print(f"Skipping {file_name} (already processed)")
                continue
            # image_id = os.path.splitext(file_name)[0]
            image_path = os.path.join(folder_path, file_name)

            img = PIL.Image.open(image_path)

            json_response = get_llm_response(file_name, img)
            print(json_response)

            process_invoices(file_name, folder_path, excel_file, json_response)


In [11]:
if __name__ == "__main__":
  main()

Mounted at /content/drive
Existing image IDs: {'IMG_6200.jpg', 'IMG_6357.jpg'}
Skipping IMG_6357.jpg (already processed)
Skipping IMG_6200.jpg (already processed)
